# Load stuff

In [ ]:
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AffinityPropagation
import colorsys
%matplotlib inline

# Functions

In [ ]:
def hexencode(rgb):
    """Transform an RGB tuple to a hex string (html color)"""
    r=rgb[0]
    g=rgb[1]
    b=rgb[2]
    return '#%02x%02x%02x' % (r,g,b)

# Simple test

![palette](img/color_palettes_23_0.png)

In [ ]:
im = Image.open('img/color_palettes_23_0.png')
w, h = im.size
colors = im.getcolors(w*h)

In [ ]:
# Check that the sum of colors match the number of pixels
assert sum([colors[i][0] for i in range(len(colors))]) == w*h

In [ ]:
for idx, c in enumerate(colors):
    plt.bar(idx, c[0], color=hexencode(c[1]), lw=1, ec='k')

# Bricks

## Load image and create histogram

In [ ]:
# im = Image.open('img/bricks.png')
im = Image.open('img/bricks-scaled.png')
w, h = im.size
colors = im.getcolors(w*h)

![bricks](img/bricks-scaled.png)

In [ ]:
# Check that the sum of colors match the number of pixels
assert sum([colors[i][0] for i in range(len(colors))]) == w*h

In [ ]:
plt.figure(figsize=(14,6))
for idx, c in enumerate(colors):
    plt.bar(idx, c[0], color=hexencode(c[1]), lw=0, width=1)
plt.axis([0, 8460, 0, 60])

## Plot individual colors

In [ ]:
df = pd.DataFrame(data={'pixels':[colors[i][0] for i in range(len(colors))],
                        'R':[colors[i][1][0] for i in range(len(colors))],
                        'G':[colors[i][1][1] for i in range(len(colors))],
                        'B':[colors[i][1][2] for i in range(len(colors))],
                        'alpha':[colors[i][1][3] for i in range(len(colors))],
                       'hex':[hexencode(colors[i][1]) for i in range(len(colors))]})

In [ ]:
plt.figure(figsize=(14, 6))
plt.subplot(121)
plt.scatter(x=df.R, y=df.G, s=30, c=df.hex, alpha=.6, edgecolor='k', lw=0.3)
plt.axis([0, 255, 0, 255])
plt.xlabel('Red', fontsize=14)
plt.ylabel('Green', fontsize=14)
plt.subplot(122)
plt.scatter(x=df.G, y=df.B, s=40, c=df.hex, alpha=.6, edgecolor='k', lw=0.3)
plt.axis([0, 255, 0, 255])
plt.xlabel('Green', fontsize=14)
plt.ylabel('Blue', fontsize=14)

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

x = df.R
y = df.G
z = df.B
c = df.hex
s = 30

ax.scatter(x, y, z, c=c, s=s, alpha=.6, edgecolor='k', lw=0.3)

ax.set_xlim3d(0, 255)
ax.set_ylim3d(0, 255)
ax.set_zlim3d(0, 255)

ax.set_xlabel('Red', fontsize=14)
ax.set_ylabel('Green', fontsize=14)
ax.set_zlabel('Blue', fontsize=14)

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

x = df.R
y = df.G
z = df.B
c = df.hex
s = df.pixels * 15

ax.scatter(x, y, z, c=c, s=s, alpha=.6, edgecolor='k', lw=0.3)

ax.set_xlim3d(0, 255)
ax.set_ylim3d(0, 255)
ax.set_zlim3d(0, 255)

ax.set_xlabel('Red', fontsize=14)
ax.set_ylabel('Green', fontsize=14)
ax.set_zlabel('Blue', fontsize=14)

## Clustering: k-means

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0, n_init=10).fit(df[['R', 'G', 'B']])

In [ ]:
df['kcenter'] = kmeans.labels_

Calculate weighted average of RBG color

In [ ]:
avg_col = np.zeros((kmeans.n_clusters, 3))
for c in range(kmeans.n_clusters):
    temp_df = df[df.kcenter == c]
    avg_col[c, 0] = np.average(temp_df.R, weights=temp_df.pixels)
    avg_col[c, 1] = np.average(temp_df.B, weights=temp_df.pixels)
    avg_col[c, 2] = np.average(temp_df.G, weights=temp_df.pixels)

Calculate weighted average of HSV color

In [ ]:
hsv_matrix = np.zeros((len(df), 3))

for i in range(len(df)):
    hsv_matrix[i] = colorsys.rgb_to_hsv(r=df.R[i], g=df.G[i], b=df.B[i])
    
df['h'] = hsv_matrix[:, 0]
df['s'] = hsv_matrix[:, 1]
df['v'] = hsv_matrix[:, 2]

avg_col2 = np.zeros((kmeans.n_clusters, 3))
for c in range(kmeans.n_clusters):
    temp_df = df[df.kcenter == c]
    avg_col2[c, 0], avg_col2[c, 1], avg_col2[c, 2] = colorsys.hsv_to_rgb(h=np.average(temp_df.h, weights=temp_df.pixels),
                                                                         s=np.average(temp_df.s, weights=temp_df.pixels),
                                                                         v=np.average(temp_df.v, weights=temp_df.pixels))

In [ ]:
fig = plt.figure(figsize=(16, 6))
ax = fig.add_subplot(131, projection='3d')

x = kmeans.cluster_centers_[:, 0]
y = kmeans.cluster_centers_[:, 1]
z = kmeans.cluster_centers_[:, 2]
c = [hexencode(kmeans.cluster_centers_[i,:]) for i in range(kmeans.n_clusters)]
s = 300

ax.scatter(x, y, z, c=c, s=s, alpha=1, edgecolor='k', lw=1)

ax.set_xlim3d(0, 255)
ax.set_ylim3d(0, 255)
ax.set_zlim3d(0, 255)
ax.set_xlabel('Red', fontsize=14)
ax.set_ylabel('Green', fontsize=14)
ax.set_zlabel('Blue', fontsize=14)
ax.set_title('k-means', fontsize=16)


ax = fig.add_subplot(132, projection='3d')
x = avg_col[:, 0]
y = avg_col[:, 1]
z = avg_col[:, 2]
c = [hexencode(r) for r in avg_col]
s = 300

ax.scatter(x, y, z, c=c, s=s, alpha=1, edgecolor='k', lw=1)

ax.set_xlim3d(0, 255)
ax.set_ylim3d(0, 255)
ax.set_zlim3d(0, 255)
ax.set_xlabel('Red', fontsize=14)
ax.set_ylabel('Green', fontsize=14)
ax.set_zlabel('Blue', fontsize=14)
ax.set_title('RGB mean', fontsize=16)


ax = fig.add_subplot(133, projection='3d')
x = avg_col2[:, 0]
y = avg_col2[:, 1]
z = avg_col2[:, 2]
c = [hexencode(r) for r in avg_col2]
s = 300

ax.scatter(x, y, z, c=c, s=s, alpha=1, edgecolor='k', lw=1)

ax.set_xlim3d(0, 255)
ax.set_ylim3d(0, 255)
ax.set_zlim3d(0, 255)
ax.set_xlabel('Red', fontsize=14)
ax.set_ylabel('Green', fontsize=14)
ax.set_zlabel('Blue', fontsize=14)
ax.set_title('HSV mean', fontsize=16)

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

x = df.R
y = df.G
z = df.B
c = df.hex
s = 30
ax.scatter(x, y, z, c=c, s=s, alpha=.6, edgecolor='k', lw=0.3)


x = kmeans.cluster_centers_[:, 0]
y = kmeans.cluster_centers_[:, 1]
z = kmeans.cluster_centers_[:, 2]
c = [hexencode(kmeans.cluster_centers_[i,:]) for i in range(kmeans.n_clusters)]
s = 1600
ax.scatter(x, y, z, c=c, s=s, alpha=1, edgecolor='k', lw=1, marker='o')

ax.set_xlim3d(0, 255)
ax.set_ylim3d(0, 255)
ax.set_zlim3d(0, 255)

ax.set_xlabel('Red', fontsize=14)
ax.set_ylabel('Green', fontsize=14)
ax.set_zlabel('Blue', fontsize=14)